# Word Embedding Techniques using Embedding Layer in Keras

In [1]:
from tensorflow.keras.preprocessing.text import one_hot

In [4]:
df = pd.read_excel(r'C:\Users\skand\Downloads\interns-tagging_Skandha.xlsx')

In [3]:
import re
import sqlite3
import pandas as pd
import numpy as np
import nltk
import tqdm as tqdm
import string
from nltk.corpus import stopwords
stop = stopwords.words("english")
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
english_stemmer=nltk.stem.SnowballStemmer('english')


from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from math import floor,ceil

from sklearn.svm import LinearSVC

from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
import seaborn as sns

##  Data Pre Processing

In [5]:
def cleaning( review, remove_stopwords=True):
   
    

    review_text = re.sub("[^a-zA-Z]"," ", review)
   
    words = review_text.lower().split()
    
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

    b=[]
    stemmer = english_stemmer 
    for word in words:
        b.append(stemmer.stem(word))

    
    return(b)

## Cleaning

In [6]:
clean_Text = []
for review in df['phrase']:
    clean_Text.append( " ".join(cleaning(review)))
    
clean_summary = []
for review in df['common idea']:
    clean_summary.append( " ".join(cleaning(review)))

In [15]:
sent = df['phrase']

In [13]:
### Vocabulary size
voc_size=10000

## One Hot Representation


In [16]:
onehot_repr=[one_hot(words,voc_size)for words in sent] 
print(onehot_repr)

[[5618, 6132, 494, 704, 2952, 7344, 676, 4952], [9005, 68, 909, 494, 704, 7296, 6812, 903, 4107], [4737, 7627, 4963, 6132, 7785, 704, 5474, 5564, 9005, 6812, 1513], [9005, 3907, 2386, 5317, 3226, 4469, 7785, 704], [4737, 7627, 4963, 6132, 7785, 704, 1798, 7785, 704, 2069, 494, 3359, 527], [9005, 4812, 4963, 6132, 8477, 1858], [4737, 6132, 7785, 704, 1798, 9434, 1798, 6413], [5618, 6833, 6132, 7785, 704, 2952], [7296, 2994, 4991, 6132, 494, 704, 3822, 6812, 5317, 7534], [9954, 4963, 6132, 6812, 704], [100, 6132, 7785, 1858], [388, 909, 7785, 704], [9005, 68, 909, 7785, 704, 7296, 2264], [9005, 68, 909, 7785, 704, 7296, 6812, 4357], [9005, 4931, 7785, 704, 7296, 6812, 778, 9675], [68, 909, 6812, 132, 8289, 4107], [9005, 4931, 7296, 4107, 5138, 8289], [9005, 68, 909, 7296, 2282, 3967, 8289], [9005, 68, 909, 8246, 7296, 8289], [5272, 4963, 7627, 909], [5564, 9005, 2315, 6132], [9005, 4931, 7296, 7333, 8289], [9005, 4931, 7333, 9675], [9005, 4931, 266, 5179, 5619], [68, 909, 6462, 7296, 828

## Word Embedding Represntation

In [17]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

In [18]:
import numpy as np

In [19]:
sent_length=30
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 7344  676 4952]
 [   0    0    0 ... 6812  903 4107]
 [   0    0    0 ... 9005 6812 1513]
 ...
 [   0    0    0 ... 8500 6339 3410]
 [   0    0    0 ... 5382 3923 4151]
 [   0    0    0 ... 5382 4828 5758]]


In [20]:
dim = 10

In [21]:
model=Sequential()
model.add(Embedding(voc_size,10,input_length=sent_length))
model.compile('adam','mse')

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 30, 10)            100000    
                                                                 
Total params: 100,000
Trainable params: 100,000
Non-trainable params: 0
_________________________________________________________________


In [23]:
print(model.predict(embedded_docs))

[[[-3.0655110e-02 -2.5146747e-02  4.6095181e-02 ... -1.7204653e-02
    2.5390871e-03  2.6136089e-02]
  [-3.0655110e-02 -2.5146747e-02  4.6095181e-02 ... -1.7204653e-02
    2.5390871e-03  2.6136089e-02]
  [-3.0655110e-02 -2.5146747e-02  4.6095181e-02 ... -1.7204653e-02
    2.5390871e-03  2.6136089e-02]
  ...
  [-4.1535426e-02 -3.0096507e-02  4.3115739e-02 ... -3.0879701e-02
    1.3833176e-02 -3.5771228e-02]
  [-4.9107280e-02 -3.0256927e-02 -2.6706744e-02 ...  3.7804972e-02
   -3.7223637e-02  1.9535985e-02]
  [-1.7620467e-02  2.4614107e-02  3.7394132e-02 ... -3.1531654e-02
    2.2111025e-02  4.4751015e-02]]

 [[-3.0655110e-02 -2.5146747e-02  4.6095181e-02 ... -1.7204653e-02
    2.5390871e-03  2.6136089e-02]
  [-3.0655110e-02 -2.5146747e-02  4.6095181e-02 ... -1.7204653e-02
    2.5390871e-03  2.6136089e-02]
  [-3.0655110e-02 -2.5146747e-02  4.6095181e-02 ... -1.7204653e-02
    2.5390871e-03  2.6136089e-02]
  ...
  [ 4.6874974e-02 -2.2259129e-02 -3.4245148e-02 ... -3.1796515e-02
   -2.3011